# `mps_tomo`

Welcome to `mps_tomo`, our entry for Qiskit Camp Asia 2019. In this example, we'll show how to perform quantum tomography using matrix product states according to [Cramer, M. *et al*. Nat Commun **1**, 149 (2010)](https://doi.org/10.1038/ncomms1147).

Let's set some parameters:

In [1]:
import numpy as np
# from mps_tomo.utils import fidelity, pauli_group

NUM_QUBITS = 5       #
K = 2                # Size of the sliding window. Related to the entanglement in the state.
DEPOLARISATION = 0.1 # Amount of depolarisation to add to the state.

This approach takes `(N - K + 1) 2^N` measurements as opposed to `3^N` in 

Now we'll create a depolarised W-state:

In [2]:
dim = 2 ** NUM_QUBITS
state = np.zeros(dim, dtype=np.complex128)
for i in range(NUM_QUBITS):
    state[1 << i] = 1
state /= np.linalg.norm(state)

dens_mat = np.outer(state, state.conj())
dens_mat = (
    DEPOLARISATION * (1 / dim) * np.eye(dim) + (1 - DEPOLARISATION) * dens_mat
)
purity = np.abs(np.trace(dens_mat @ dens_mat))

print(f"purity = {purity}")

purity = 0.8159374999999996


We'll now calculate the reduced density matrices of the sliding windows of qubits. Experimentally, these would be determined via multiple Pauli product measurements.

In [3]:
def reduce(qubit):
    left_size = 2 ** qubit
    reduced_size = 2 ** K
    right_size = np.size(dens_mat, axis=0) // (left_size * reduced_size)

    reshaped = np.reshape(
        dens_mat,
        (left_size, reduced_size, right_size, left_size, reduced_size, right_size),
    )

    return np.einsum("aibajb->ij", reshaped)

sigmas = (reduce(q) for q in range(NUM_QUBITS - K + 1))

We now perform the iterative process to give a best matrix product state estimate:

In [4]:
from mps_tomo.uncert import iteration

y_vec = iteration(K, sigmas, NUM_QUBITS, max_its=100, delta=0.1)
y_dens = np.outer(y_vec, y_vec.conj())

We can now see the overlap with the original W-state, and the fidelity with its depolarised density matrix.

In [5]:
from mps_tomo.utils import fidelity

overlap = np.abs(state.conj() @ y_vec) ** 2
fid = fidelity(y_dens, dens_mat)

print(f"overlap = {overlap}")
print(f"fidelity = {fid}")

overlap = 0.9999930242779583
fidelity = 0.9031187286941512
